# Alpha–Phi Binary Explorer (v9d) — *Ultra‑fine a‑grid (1.37×10⁻⁸ step)*

* `a` offsets step **1.370359 × 10⁻⁸** (≈ one part per 70 million)  
  * Range: ±5×10⁻⁵ around `log10(α)` → 7 300 points.  
* `b` values trimmed to 0.97 … 0.995 (four points) to keep combos ~29 k.  
* Iterations: **600** (because b≈0.995 converges slowly).  
* Skips early if `rel_err > 0.5`.  Stops on first < 1 % hit.  Logs to `log_ultrafine.csv`.

In [1]:
import mpmath as mp, csv, math
mp.mp.dps = 300
ALPHA = mp.mpf('7.2973525693e-3')
log_alpha = mp.log10(ALPHA)

def fib_word_bits(n):
    a,b='0','01'
    while len(b)<n:
        a,b=b,b+a
    return b[:n]

def fib_word_real(n):
    bits=fib_word_bits(n)
    return mp.fsum(int(b)*mp.mpf(2)**(-(k+1)) for k,b in enumerate(bits))


In [2]:
def iterate_log(y0, a, b, steps=600):
    for _ in range(steps):
        y0 = a - b*y0
    return y0

## Ultra‑fine sweep

In [3]:
step = mp.mpf('1.370359e-8')
a_vals = [log_alpha + (i*step) for i in range(-5000,5001)]  # ±5e-5
b_vals = [mp.mpf(str(b)) for b in (0.97,0.98,0.99,0.995)]
with open('log_ultrafine.csv','w',newline='') as f:
    w=csv.writer(f); w.writerow(['a','b','bits','rel_err'])
    for bits in (512,):
        y0 = mp.log10(fib_word_real(bits))
        for a in a_vals:
            for b in b_vals:
                y = iterate_log(y0,a,b,600)
                x = mp.power(10,y)
                rel = abs((x-ALPHA)/ALPHA)
                if rel > 0.5:
                    continue
                if rel < 0.01:
                    w.writerow([str(a),str(b),bits,str(rel)])
                    print('Hit <1% ', mp.nstr(rel,8), 'params', (a,b,bits))
                    raise SystemExit
print('No hit <1% in ultra‑fine grid')


No hit <1% in ultra‑fine grid
